In [ ]:
import jwt
import json
import time

from functools import wraps
from decouple import Config, RepositoryEnv
from fastapi import HTTPException, Request, WebSocket
from keycloak import KeycloakOpenID
from jwcrypto.jwt import JWTExpired
from jwcrypto.jws import InvalidJWSObject, InvalidJWSSignature
from datetime import datetime

KEYCLOAK_URL = "http://localhost:8080"
KEYCLOAK_REALM_NAME = "team_online"
KEYCLOAK_BACKEND_CLIENT_ID = "benyon_be"
KEYCLOAK_BACKEND_CLIENT_SECRET = "BC4FFJYDUBH1ccuPyB3sRvWSZblrs4SZ"

async def keycloak_verif(token:str):
    keycloak_openid = KeycloakOpenID(server_url=KEYCLOAK_URL,
                                    client_id=KEYCLOAK_BACKEND_CLIENT_ID,
                                    realm_name=KEYCLOAK_REALM_NAME,
                                    client_secret_key=KEYCLOAK_BACKEND_CLIENT_SECRET
                                    )

    dec_tok = await keycloak_openid.a_decode_token(token, validate=True)
    intr_tok = await keycloak_openid.a_introspect(token)
    if intr_tok.get("active"): 
        if time.time()>intr_tok.get('exp'): raise Exception("auth token expired")
    else:
        raise Exception("inactive auth token")
    
    auth_status = keycloak_openid.uma_permissions(token)
    permissions = [permissions_dict[i] for permissions_dict in auth_status for i in permissions_dict if i=="rsname"]
    

    return intr_tok, permissions

token_from_request = ""
intr_tok, permissions = await keycloak_verif(token_from_request)
print(permissions)

['api_endpoint_create', 'Default Resource', 'api_endpoint_delete', 'api_endpoint_upload']
